## Review Analysis using RNN

Sentiment Analysis of movie reviews using Recurrent Neural Networks and LSTM.

Importing the dataset

In [106]:
import tensorflow as tf
import numpy as np
import time
from collections import Counter

In [107]:
reviews = ''
labels = ''

with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
    
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [108]:
print ('Sample Positive Review-->')
print (reviews.split('\n')[0])

Sample Positive Review-->
bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   


In [109]:
print ('Sample Negative Review-->')
print (reviews.split('\n')[1])

Sample Negative Review-->
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turned into an insane  violent mob by the crazy chantings of it  s singers . unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting . even those from the era should be turned off . the cryptic dialogue would make shakespeare seem easy to a third grader . on a technical level it  s better than you might think with some good cinematography by future great vilmos zsigmond . future stars sally kirkland and frederic forrest can be seen briefly .  


In [110]:
len(labels.split('\n'))

25001

## Some data analysis

In [111]:
from string import punctuation

all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')
labels = labels.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [114]:
total_words = len(reviews.split())
total_characters = len(reviews)
unique_words = len(set(reviews.split()))
unique_characters = len(set(reviews))
total = len(reviews)

print ('FOR POSITIVE REVIEWS')
print ("Total words :", total_words)
print ("Total characters :", total_characters)
print ("Unique words :", unique_words)
print ("Unique characters:", unique_characters)
print ('Total: ', total)

AttributeError: 'list' object has no attribute 'split'

In [ ]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [ ]:
for i in range(len(reviews)):
    if(labels[i] == 'positive'):
        for word in reviews[i].split():
            positive_counts[word] += 1
            total_counts[word] += 1
    else:
        for word in reviews[i].split():
            negative_counts[word] += 1
            total_counts[word] += 1

In [ ]:
positive_counts.most_common()[:20]

In [113]:
negative_counts.most_common()[:20]

[('', 537725),
 ('the', 127180),
 ('a', 62576),
 ('and', 58288),
 ('of', 54119),
 ('to', 54011),
 ('br', 40613),
 ('is', 39414),
 ('.', 38792),
 ('i', 37910),
 ('it', 37694),
 ('in', 34267),
 ('this', 32238),
 ('that', 29620),
 ('s', 25389),
 ('was', 20757),
 ('movie', 19711),
 ('for', 17157),
 ('but', 17157),
 ('t', 16691)]

In [87]:
pos_neg_ratios = Counter()
for term,cnt in list(total_counts.most_common()):
    if(cnt > 100):
        pos_neg_ratio = positive_counts[term] / float(negative_counts[term]+1)
        pos_neg_ratios[term] = pos_neg_ratio

In [88]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 6.000566122298142
Pos-to-neg ratio for 'amazing' = 16.114427860696516
Pos-to-neg ratio for 'terrible' = 2.8245462402765775


In [89]:
for word,ratio in pos_neg_ratios.most_common():
    pos_neg_ratios[word] = np.log(ratio)

In [90]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 1.79185381849338
Pos-to-neg ratio for 'amazing' = 2.7797150111122533
Pos-to-neg ratio for 'terrible' = 1.038347728786217


In [92]:
pos_neg_ratios.most_common()[:30]

[('edie', 6.0591231955817966),
 ('antwone', 5.575949103146316),
 ('goldsworthy', 5.5606816310155276),
 ('rukh', 5.5373342670185366),
 ('timon', 5.4889377261566867),
 ('rea', 5.3375380797013179),
 ('visconti', 5.3082676974012051),
 ('giovanna', 5.2626901889048856),
 ('blandings', 5.2574953720277815),
 ('kells', 5.2574953720277815),
 ('gino', 5.181783550292085),
 ('excellently', 5.1416635565026603),
 ('minnelli', 5.1298987149230735),
 ('lindy', 5.1119877883565437),
 ('din', 5.0998664278241987),
 ('ossessione', 5.0498560072495371),
 ('pumbaa', 5.0434251169192468),
 ('caruso', 5.0106352940962555),
 ('paulie', 4.990432586778736),
 ('ahmad', 4.9767337424205742),
 ('vonnegut', 4.9767337424205742),
 ('gackt', 4.9767337424205742),
 ('khouri', 4.9698132995760007),
 ('sabu', 4.9126548857360524),
 ('gunga', 4.8828019225863706),
 ('beckham', 4.8441870864585912),
 ('gundam', 4.836281906951478),
 ('matthau', 4.824305715904762),
 ('jaffar', 4.8202815656050371),
 ('fagin', 4.8121843553724171)]

Encoding words to integers. Buidling a dictionary to convert words to integers.

In [94]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])

In [103]:
vocab_to_int

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'it': 8,
 'in': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 's': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 'you': 21,
 'on': 22,
 't': 23,
 'not': 24,
 'he': 25,
 'are': 26,
 'his': 27,
 'have': 28,
 'be': 29,
 'one': 30,
 'all': 31,
 'at': 32,
 'they': 33,
 'by': 34,
 'an': 35,
 'who': 36,
 'so': 37,
 'from': 38,
 'like': 39,
 'there': 40,
 'her': 41,
 'or': 42,
 'just': 43,
 'about': 44,
 'out': 45,
 'if': 46,
 'has': 47,
 'what': 48,
 'some': 49,
 'good': 50,
 'can': 51,
 'more': 52,
 'she': 53,
 'when': 54,
 'very': 55,
 'up': 56,
 'time': 57,
 'no': 58,
 'even': 59,
 'my': 60,
 'would': 61,
 'which': 62,
 'story': 63,
 'only': 64,
 'really': 65,
 'see': 66,
 'their': 67,
 'had': 68,
 'we': 69,
 'were': 70,
 'me': 71,
 'well': 72,
 'than': 73,
 'much': 74,
 'get': 75,
 'bad': 76,
 'been': 77,
 'people': 78,
 'will': 79,
 'do': 80,
 'other': 81,
 'also': 82,
 'into':

In [104]:
review[0]

NameError: name 'review' is not defined

In [116]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 2470


In [117]:
seq_len = 300
features = np.zeros((len(reviews_ints), seq_len), dtype=int)
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [119]:
features[:10,:10]

array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [106968,    906, 106969,     14,    776,  33264, 106970,     38,
            71,     32],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [   133,     10, 106998,   6469,   1962,   1145,    463,     19,
           319,   4731],
       [   390,  12057, 107014,  25980,  48110,   2698,   9265,   7796,
         27965,   4226],
       [   255,   1358,   2698,   9265,   1248,  13200,    771,      6,
          1337,   1317]])

In [120]:
split_frac = 0.8
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(19200, 300) 
Validation set: 	(2400, 300) 
Test set: 		(2400, 300)


In [121]:
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.001

In [148]:
n_words = len(vocab_to_int) + 1 # Adding 1 because we use 0's for padding, dictionary started at 1

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None,], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [123]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [125]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [126]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                             initial_state=initial_state)

In [127]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [128]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [129]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [144]:
x, y=next(get_batches(train_x, train_y),1)
len(y)

100

In [150]:
epochs = 10

#with graph.as_default():
#    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y,
                    keep_prob: 0.8,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y,
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

ValueError: Fetch argument <tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(500, 256) dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0", shape=(500, 256), dtype=float32) is not an element of this graph.)